In [1]:
import numpy as np
import pandas as pd
import keras
import cv2
from matplotlib import pyplot as plt
import os
import random
from PIL import Image

In [2]:
df = pd.read_csv('train.csv')

In [ ]:
df

In [9]:
samples = 20000
df = df[df["id"].astype(str).str.startswith('00')]
num_classes = len(df["landmark_id"].unique())
num_data = len(df)

In [10]:
num_classes

13589

In [11]:
num_data

16157

In [ ]:
data = pd.DataFrame(df["landmark_id"].value_counts())

data.reset_index(inplace=True)
data.head()

In [ ]:
data.tail()

In [14]:
data.columns=['index','landmark_id']

In [ ]:
data['index'].describe()

In [ ]:
plt.hist(data['landmark_id'],100,range=(0,58),label='test')

In [17]:
data['landmark_id'].between(0,5).sum()

13549

In [18]:
data['landmark_id'].between(5,10).sum()

69

In [ ]:
plt.hist(df["landmark_id"], bins=sorted(df["landmark_id"].unique()))
# Sort the unique values to ensure monotonically increasing bins

In [ ]:
from sklearn.preprocessing import LabelEncoder
lencoder = LabelEncoder()
lencoder.fit(df["landmark_id"])


In [ ]:
df.head()

In [33]:
def encode_label(label):
    return lencoder.transform(lbl)

In [34]:
def decode_label(label):
    return lencoder.inverse_transform(lbl)

In [35]:

def get_image_from_number(num):
    fname, label = df.loc[num, :]
    fname = fname + '.jpg'
    f1 = fname[0]
    f2 = fname[1]
    f3 = fname[2]
    path = os.path.join(f1, f2, f3, fname)
    im = cv2.imread(os.path.join(base_path, path))
    return im, label



In [ ]:
from keras.applications.vgg19 import VGG19
from keras.layers import *
from keras import Sequential
tf.compat.v1.disable_eager_execution()

In [ ]:
learning_rate = 0.0001
decay_speed = 1e6
momentum = 0.09
loss_function = "sparse_categorical_crossentropy"
source_model = VGG19(weights=None)
drop_layer = Dropout(0.5)
drop_layer2 = Dropout(0.5)

In [ ]:
model - Sequential()
for layer in source_nodel. layers[:-1]:
  if layer == source_nodel.layers[- 25]:
     model. add (Batchilornalization())
  model.add(layer)
nodel. add(Dense(nun_classes, activation « "softnax"))
model. summary

In [ ]:
optim1 = keras.optimizer_v1.RMSprop(Ir = learning_rate)
model. conplle(optimizer-optima,loss-loss_function,
netries - ("accuracy"))

In [ ]:
def image_reshape(im, target_size):
return cv2.resize(im, target_size)

In [ ]:
import numpy as np

def get_batch(dataframe, start, batch_size):
    """
    Fetches a batch of images and labels from the dataframe.

    Args:
        dataframe (pd.DataFrame): Dataframe containing image IDs and labels.
        start (int): Starting index for the batch.
        batch_size (int): Size of the batch.

    Returns:
        Tuple[np.ndarray, np.ndarray]: A tuple containing the image array and label array.
    """
    image_array = []
    label_array = []
    end_ing = start + batch_size  # Corrected variable name and calculation

    if end_ing > len(dataframe):  # Corrected condition
        end_ing = len(dataframe)  # Corrected assignment

    for idx in range(start, end_ing):  # Corrected loop variable name
        im, label = get_image_from_number(idx, dataframe)  # Corrected function call and variable names
        im = image_reshape(im, (224, 224)) / 255.0  # Corrected function call and variable name
        image_array.append(im)  # Corrected variable name
        label_array.append(label)  # Corrected variable name

    label_array = encode_label(label_array)  # Corrected function call and variable name
    return np.array(image_array), np.array(label_array)  # Corrected variable names

In [ ]:
batch_size = 16
epoch_shuffle = True
weight_classes = True
epochs - 1
# split
train, val - np.split(df.sanple(frac-1),[Int(0.8*len(df))])
print(len(train) )
print(len(val))

In [ ]:
for e in range(epochs) :
   print("Epoch:" • str (er1) + */™» str(epochs))
     if epoch_shuffle:
        train = train.sanple(frac- 1)
    for It in range(int(np.cell(len(train)/batch_size))):
        X train, Y_train - get_batch(train, It*batch_size, batch_size)
        model. train _on_batch(X_train, y_train)
model, save ("Model" )

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

# ... (Assuming get_batch, get_image_from_number, and model are defined) ...

batch_size = 16
errors = 0  # Initialize errors to 0
good_preds = []
bad_preds = []

for it in range(int(np.ceil(len(val) / batch_size))):  # Corrected np.cell to np.ceil
    X_val, Y_val = get_batch(val, it * batch_size, batch_size)  # Corrected It to it
    result = model.predict(X_val)  # Assuming 'model' is your trained model
    cla = np.argmax(result, axis=1)  # Corrected axis-1 to axis=1

    for idx, res in enumerate(result):  # Corrected 1dx to idx
        if cla[idx] == Y_val[idx]:  # Corrected y_val to Y_val and 1 to ==
            good_preds.append([batch_size * it + idx, cla[idx], res[cla[idx]]])  # Corrected 1dx to idx
        else:
            errors += 1  # Increment errors for incorrect predictions
            bad_preds.append([batch_size * it + idx, cla[idx], res[cla[idx]]])  # Corrected 1dx to idx

# Display some good predictions
for i in range(min(5, len(good_preds))):  # Limit to 5 or fewer predictions
    n = good_preds[i][0]  # Get image index from good_preds
    img, lbl = get_image_from_number(n, val)  # Get image and label
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for plt.imshow
    plt.imshow(img)
    plt.title(f"Prediction: {good_preds[i][1]}, Actual: {lbl}")  # Add title with prediction and actual label
    plt.show()

print(f"Total Errors: {errors}")